In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('eth_addresses.csv')

In [4]:
df.head()

,Address,Name,Account Type,Contract Type,Entity,Label,Tags,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,0x8ab7404063ec4dbcfd4598215992dc3f8ec853d7,Akropolis (AKRO),Smart Contract,Token,DeFi,Legit,DeFi,Token Contract,NaN,NaN,NaN,NaN
1,0x1c74cff0376fb4031cd7492cd6db2d66c3f2c6b9,bZx Protocol Token (BZRX),Smart Contract,Token,DeFi,Legit,Token Contract,bZx,DeFi,0x Ecosystem,Trading,Liquidity
2,0x06af07097c9eeb7fd685c692751d5c66db49c215,Chai (CHAI),Smart Contract,Token,DeFi,Legit,DeFi,Token Contract,Verified Contract,NaN,NaN,NaN
3,0xc00e94cb662c3520282e6f5717214004a7f26888,Compound (COMP),Smart Contract,Token,DeFi,Legit,DAO,Compound,Token Contract,NaN,NaN,NaN
4,0xb3319f5d18bc0d84dd1b4825dcde5d5f7266d407,Compound 0x (cZRX),Smart Contract,Token,DeFi,Legit,Token Contract,DeFi,Compound,0x Ecosystem,NaN,NaN


In [5]:
new_data_1 = pd.read_csv('EthRich.csv')

In [6]:
new_data_2 = pd.read_csv('accounts.csv')

In [7]:
new_data_1.head()

,0x00000000219ab540356cbb839cbe05303d7705fa
0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
1,0xda9dfa130df4de4673b89022ee50ff26f6ea73cf
2,0xbe0eb53f46cd790cd13851d5eff43d12404d33e8
3,0x73bceb1cd57c711feac4224d062b0f6ff338501e
4,0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5


In [8]:
new_data_2.head()

,address,chainId,label,nameTag
0,0x01d9f4d104668cdc0b6d13c45dff5e15d58d8f28,1,0x-protocol,0x: MultiSig 2
1,0x06adaff9c864eb59ade310bc21150a8f4d5d4ab4,1,0x-protocol,0x: WBTC Merchant Deposit Address
2,0x080bf510fcbf18b91105470639e9561022937712,1,0x-protocol,0x: Exchange v2.1
3,0x0ac2d6f5f5afc669d3ca38f830dad2b4f238ad3f,1,0x-protocol,0x: Eth2Dai Bridge
4,0x12459c951127e0c374ff9105dda097662a027093,1,0x-protocol,0x: Exchange v1


In [11]:
combined_df = pd.read_csv('combined_wallets.csv')

In [32]:
print(new_data_1.columns)
print(new_data_2.columns)
print(combined_df.columns)


Index(['0x00000000219ab540356cbb839cbe05303d7705fa'], dtype='object')
Index(['address', 'chainId', 'label', 'nameTag'], dtype='object')
Index(['Address', 'Name', 'isSafe', 'Tags'], dtype='object')


In [35]:
# Standardize column names across datasets

# Rename column in new_data_1 to "Address" (it has a single unnamed column with addresses)
new_data_1.rename(columns={new_data_1.columns[0]: 'Address'}, inplace=True)

# Rename relevant columns in new_data_2 to align with the format
new_data_2.rename(columns={'address': 'Address', 'nameTag': 'Name', 'label': 'Tags'}, inplace=True)

# Keep only necessary columns in new_data_2
new_data_2 = new_data_2[['Address', 'Name', 'Tags']]

# Load combined_df and ensure column consistency
try:
    combined_df = pd.read_csv('combined_wallets.csv')
    if 'address' in combined_df.columns:
        combined_df.rename(columns={'address': 'Address'}, inplace=True)
except FileNotFoundError:
    print("Error: combined_wallets.csv not found.")

# Drop NaN values in Address column for all datasets before merging
new_data_1.dropna(subset=['Address'], inplace=True)
new_data_2.dropna(subset=['Address'], inplace=True)

# Select random samples from both datasets
new_data_1_sampled = new_data_1.sample(n=15000, random_state=42)
new_data_2_sampled = new_data_2.sample(n=38000, random_state=42)

# Merge the sampled datasets
merged_new_data = pd.concat([new_data_1_sampled, new_data_2_sampled], ignore_index=True)

# Assign default values for missing fields
merged_new_data['isSafe'] = True  # Assume newly added addresses are safe
merged_new_data['Name'] = merged_new_data['Name'].fillna('NA')  # Fill missing names with 'NA'
merged_new_data['Tags'] = merged_new_data['Tags'].fillna('NA')  # Fill missing tags with 'NA'

# Merge with combined_df
final_combined_df = pd.concat([combined_df, merged_new_data], ignore_index=True)

# Shuffle the final dataset and reset index
final_combined_df = final_combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the updated dataset
final_combined_df.to_csv('final_combined_wallets_fixed.csv', index=False)

In [36]:
final_combined_df.head()

,Address,Name,isSafe,Tags
0,0xe1afe1fd76fd88f78cbf599ea1846231b8ba3b6b,Synthetix: Proxy sDEFI Token,True,proxy-contract
1,0xf6937f486fbc7dcb0a2c689e34d1dedf20b58731,NA,True,NA
2,0xf4618a00ed016475ae4033bfff78946fec712f3b,NA,True,NA
3,0x0c3de458b51a11da7d4616f42f66c861e3859d3e,MEV Bot: 0x0c3...D3E,True,mev-bot
4,0x7b86d0136fb2a90bec18e92046c8400f2f4ab5f4,NA,True,NA


In [37]:
final_combined_df.shape

(61919, 4)

In [38]:
final_combined_df['isSafe'].value_counts()

isSafe
True     53914
False     8005
Name: count, dtype: int64

In [40]:
final_combined_df = final_combined_df.drop_duplicates(subset=['Address'], keep='first')

In [41]:
final_combined_df.to_csv('final_combined_wallets_fixed.csv', index=False)

In [57]:
final_combined_df.columns

Index(['Address', 'Name', 'isSafe', 'Tags'], dtype='object')

In [55]:
api_key = "7A32HIJRFBSUJQK8S6WD3E2XCDS6XBW9JZ"  # Replace with your actual API key

In [67]:
# dataset_path = 'final_combined_wallets_fixed.csv'
# combined_df = pd.read_csv(dataset_path)

In [68]:
# # Ensure required columns exist
# if 'transactions' not in combined_df.columns:
#     combined_df['transactions'] = None
# if 'balance' not in combined_df.columns:
#     combined_df['balance'] = None
# if 'noOfTrx' not in combined_df.columns:
#     combined_df['noOfTrx'] = 0 


In [71]:
# combined_df.columns

Index(['Address', 'Name', 'isSafe', 'Tags', 'transactions', 'balance',
       'noOfTrx_x', 'Transactions', 'noOfTrx_y', 'Balance'],
      dtype='object')

In [ ]:
import pandas as pd
import requests
import time

# Load the cleaned dataset
dataset_path = 'final_combined_wallets_fixed.csv'
combined_df = pd.read_csv(dataset_path)

# Ensure required columns exist
if 'transactions' not in combined_df.columns:
    combined_df['transactions'] = None
if 'balance' not in combined_df.columns:
    combined_df['balance'] = None
if 'noOfTrx' not in combined_df.columns:
    combined_df['noOfTrx'] = 0  # Initialize transaction count

# API configuration
api_url_trx = "https://api.etherscan.io/api"
api_url_balance = "https://api.etherscan.io/api"

# Set testing mode (limit to 10 wallets if True)
testing = False

# Extract addresses
addresses = combined_df['Address'].dropna().tolist()

# Limit addresses if in testing mode
if testing:
    addresses = addresses[:10]

# Initialize lists for results
result_transactions = []
result_balances = []

# Chunk addresses for balance API (max 5 per request)
chunks = [addresses[i:i + 5] for i in range(0, len(addresses), 5)]

# API Rate Limit Handling
MAX_CALLS_PER_SECOND = 5
SLEEP_TIME = 1 / MAX_CALLS_PER_SECOND  # Wait time to maintain 5 calls/sec limit

# Function to make API calls with error handling & rate limit compliance
def make_api_call(url, params):
    try:
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if data.get('status') == "1":
                return data['result']
            else:
                print(f"API Error: {data.get('message')}")
                return None
        else:
            print(f"HTTP Error {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        time.sleep(2)  # Wait before retrying
    time.sleep(SLEEP_TIME)  # Enforce API rate limit
    return None

# Process addresses in chunks
for i, chunk in enumerate(chunks):
    # Balance API call
    balance_params = {
        "module": "account",
        "action": "balancemulti",
        "address": ",".join(chunk),
        "tag": "latest",
        "apikey": api_key
    }
    balance_result = make_api_call(api_url_balance, balance_params)
    
    if balance_result:
        for item in balance_result:
            result_balances.append({
                "Address": item['account'],
                "Balance": item['balance']
            })

    # Transaction API call for each address (rate-limited)
    for address in chunk:
        trx_params = {
            "module": "account",
            "action": "txlist",
            "address": address,
            "startblock": 0,
            "endblock": 99999999,
            "sort": "asc",
            "apikey": api_key
        }
        trx_result = make_api_call(api_url_trx, trx_params)
        
        if trx_result:
            result_transactions.append({
                "Address": address,
                "Transactions": trx_result,
                "noOfTrx": len(trx_result)
            })
            print(f"Address {address} - Transactions Found: {len(trx_result)}")
        else:
            # Handle case where no transactions exist
            result_transactions.append({
                "Address": address,
                "Transactions": "[]",  # Empty list if no transactions exist
                "noOfTrx": 0  # Set transaction count to 0
            })
            print(f"Address {address} - No Transactions Found.")
        
        time.sleep(SLEEP_TIME)  # Ensure we do not exceed 5 calls/sec

# Convert results to DataFrames
transactions_df = pd.DataFrame(result_transactions)
balances_df = pd.DataFrame(result_balances)

# Ensure transactions_df has 'noOfTrx' before merging
if 'noOfTrx' not in transactions_df.columns:
    transactions_df['noOfTrx'] = 0

# Ensure 'noOfTrx' exists in combined_df before merging
if 'noOfTrx' not in combined_df.columns:
    combined_df['noOfTrx'] = 0

# Merge with the main dataset
combined_df = combined_df.merge(transactions_df, on='Address', how='left', suffixes=('', '_trx'))
combined_df = combined_df.merge(balances_df, on='Address', how='left', suffixes=('', '_bal'))

# Rename potential duplicate columns correctly
if 'noOfTrx_trx' in combined_df.columns:
    combined_df.rename(columns={'noOfTrx_trx': 'noOfTrx'}, inplace=True)
if 'Balance_bal' in combined_df.columns:
    combined_df.rename(columns={'Balance_bal': 'Balance'}, inplace=True)

# Fill missing transaction counts
combined_df['noOfTrx'] = combined_df['noOfTrx'].fillna(0).astype(int)

# Save the updated dataset
output_file = 'final_combined_wallets_with_trx_balances.csv'
combined_df.to_csv(output_file, index=False)

print(f"Updated dataset saved to {output_file}")


Address 0xe1afe1fd76fd88f78cbf599ea1846231b8ba3b6b - Transactions Found: 554
Address 0xf6937f486fbc7dcb0a2c689e34d1dedf20b58731 - Transactions Found: 19
Address 0xf4618a00ed016475ae4033bfff78946fec712f3b - Transactions Found: 139
Address 0x0c3de458b51a11da7d4616f42f66c861e3859d3e - Transactions Found: 10000
Address 0x7b86d0136fb2a90bec18e92046c8400f2f4ab5f4 - Transactions Found: 3
Address 0x5d234535f2283c8e48a05ad9b7994077bdc743ab - Transactions Found: 28
Address 0xde05dcb77aff940dd524c056e4219786114e37c4 - Transactions Found: 11
Address 0x24b1d27b0f6b5a2aa052acf59817a8d9e7a8600a - Transactions Found: 48
Address 0x50fb734f0448a122b102f9a211e52c2f14eb208a - Transactions Found: 10
Address 0xaf429aad2da83c748d915f5901bdec79fd933275 - Transactions Found: 1
Address 0x886fc23811240f94e32fa66c748277f8f1b4a805 - Transactions Found: 1
Address 0x93737345d0395976751329feff0fb84fb13d58ec - Transactions Found: 10
Address 0x712d0f306956a6a4b4f9319ad9b9de48c5345996 - Transactions Found: 1351
Address 

In [3]:
op_file = pd.read_csv('final_combined_wallets_with_trx_balances.csv')

In [4]:
op_file.tail(10)

,Address,Name,isSafe,Tags,transactions,balance,noOfTrx,Transactions,noOfTrx.1,Balance
52868,0x84646f736795a8bc22ab34e05c8982cd058328c7,Harvest.Finance: YFI Pool,True,yield-farming,NaN,NaN,0,"[{'blockNumber': '10776494', 'timeStamp': '159...",1044,0
52869,0x5269d0ba4afa22bc8a4845a5ec4a59c48f7b38e1,NaN,True,airdrop-hunter,NaN,NaN,0,"[{'blockNumber': '13489256', 'timeStamp': '163...",4,2169452242986000
52870,0x221dd436df380a957b0fff2359549fd0368c6594,NaN,True,NaN,NaN,NaN,0,"[{'blockNumber': '4249885', 'timeStamp': '1504...",242,34181892106133550041
52871,0xae7f458667f1b30746354abc3157907d9f6fd15e,NaN,True,NaN,NaN,NaN,0,"[{'blockNumber': '12987335', 'timeStamp': '162...",189,45425996875803531736
52872,0xD33441A44Da0c1DCEE596aBB7ebDCdBa77c7DDBd,NaN,False,NaN,NaN,NaN,0,"[{'blockNumber': '5215208', 'timeStamp': '1520...",58,11118367256799
52873,0x8aeda5b17366ebb9b9d3a4b91a1ac432c385606e,NaN,True,NaN,NaN,NaN,0,"[{'blockNumber': '4171844', 'timeStamp': '1503...",54,22714410955797866645
52874,0xd4c5420261cd40cd5521f68ab6ee7be403ee05a4,Balancer: ETH/PRQ 88.89/11.11,True,balancer,NaN,NaN,0,"[{'blockNumber': '10732113', 'timeStamp': '159...",1,0
52875,0xa659e66e116d354e779d8dbb35319af67171ffb4,Armor.fi: ARMOR/DAI Uni LP,True,armor-fi,NaN,NaN,0,"[{'blockNumber': '11715142', 'timeStamp': '161...",1162,0
52876,0x625bebea868a3053636a6719d683fd7e793fdb79,NaN,False,NaN,NaN,NaN,0,[],0,0
52877,0x9bb86ab2ebe3e6b9b580d557f7adcc4ff746ec8f,NaN,True,NaN,NaN,NaN,0,"[{'blockNumber': '8002453', 'timeStamp': '1561...",2916,32616380022379778


In [9]:
op_file.tail(1).Transactions

52877    [{'blockNumber': '8002453', 'timeStamp': '1561...
Name: Transactions, dtype: object

In [11]:
import pandas as pd


# Select relevant columns for Wallet Table
wallets_df = op_file[['Address', 'Name', 'isSafe', 'Tags', 'balance', 'noOfTrx']]

# Save the Wallets Table
wallets_df.to_csv('wallets_table.csv', index=False)
print("Wallets table saved!")


Wallets table saved!


In [15]:
import pandas as pd
import json


# Function to extract transactions
def extract_transactions(row):
    transactions = []
    if isinstance(row['Transactions'], str) and row['Transactions'] != "[]":
        trx_list = json.loads(row['Transactions'].replace("'", '"'))  # Convert string to JSON
        for trx in trx_list:
            transactions.append({
                "transaction_id": trx.get('hash', None),
                "wallet_address": row['Address'],
                "blockNumber": trx.get('blockNumber', None),
                "timeStamp": trx.get('timeStamp', None),
                "from_address": trx.get('from', None),
                "to_address": trx.get('to', None),
                "value": trx.get('value', None),
                "gasPrice": trx.get('gasPrice', None),
                "contractAddress": trx.get('contractAddress', None),
                "txreceipt_status": trx.get('txreceipt_status', None),
                "isError": trx.get('isError', None)
            })
    return transactions

# Extract transactions from each row
transactions_list = op_file.apply(extract_transactions, axis=1)

# Flatten the list of lists into a DataFrame
transactions_df = pd.DataFrame([trx for sublist in transactions_list for trx in sublist])

# Save Transactions Table
transactions_df.to_csv('transactions_table.csv', index=False)

print("🚀 Transactions table saved successfully!")


🚀 Transactions table saved successfully!


In [16]:


trx_tbl = pd.read_csv('transactions_table.csv')

In [17]:
trx_tbl.head()

,transaction_id,wallet_address,blockNumber,timeStamp,from_address,to_address,value,gasPrice,contractAddress,txreceipt_status,isError
0,0x95e2061d2e635c4ab0b30afd1b36d4e4a7f47def2f2e...,0xe1afe1fd76fd88f78cbf599ea1846231b8ba3b6b,8971531,1574298388,0xb64ff7a4a33acdf48d97dab0d764afd0f6176882,NaN,0,15100000000,0xe1afe1fd76fd88f78cbf599ea1846231b8ba3b6b,1.0,0
1,0x221d293422ef3cb497daf57a45b8de6ad9c3e48b2fcd...,0xe1afe1fd76fd88f78cbf599ea1846231b8ba3b6b,8971536,1574298449,0xb64ff7a4a33acdf48d97dab0d764afd0f6176882,0xe1afe1fd76fd88f78cbf599ea1846231b8ba3b6b,0,15100000000,NaN,1.0,0
2,0xf459b9d7eb23d875c1ed5534880612a112b50962cec9...,0xe1afe1fd76fd88f78cbf599ea1846231b8ba3b6b,8971655,1574300178,0xb64ff7a4a33acdf48d97dab0d764afd0f6176882,0xe1afe1fd76fd88f78cbf599ea1846231b8ba3b6b,0,20000000000,NaN,1.0,0
3,0x94623a3381503975011ff2fc4fba504525ac7301418b...,0xe1afe1fd76fd88f78cbf599ea1846231b8ba3b6b,9060286,1575632552,0x4d37f28d2db99e8d35a6c725a5f1749a085850a3,0xe1afe1fd76fd88f78cbf599ea1846231b8ba3b6b,0,11000000000,NaN,1.0,0
4,0x11404c537cfbbc5f3f562525010681952f54e2783216...,0xe1afe1fd76fd88f78cbf599ea1846231b8ba3b6b,9078069,1575904207,0x52a4e3cdce5d689e562291aa2d402f1541c777d9,0xe1afe1fd76fd88f78cbf599ea1846231b8ba3b6b,0,1000000000,NaN,1.0,0
